In [6]:
from pyprojroot import here
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from dotenv import load_dotenv
from urllib.parse import quote_plus
import pandas as pd
import os
from langchain_community.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

In [8]:
# Load environment variables
load_dotenv()

True

In [4]:
df = pd.read_csv('../../data/csv_xlsx/f1_pitstops_2018_2024.csv')

In [5]:
df.head(10)

,Season,Round,Circuit,Driver,Constructor,Laps,Position,TotalPitStops,AvgPitStopTime,Race Name,...,Tire Usage Aggression,Fast Lap Attempts,Position Changes,Driver Aggression Score,Abbreviation,Stint,Tire Compound,Stint Length,Pit_Lap,Pit_Time
0,2018,1,Albert Park Grand Prix Circuit,Sebastian Vettel,Ferrari,58,1,1,21.787,Australian Grand Prix,...,0.017241,44.76882,0.000000,6.755003,VET,1.0,ULTRASOFT,25.0,26.0,21.787
1,2018,1,Albert Park Grand Prix Circuit,Sebastian Vettel,Ferrari,58,1,1,21.787,Australian Grand Prix,...,0.017241,44.76882,0.000000,6.755003,VET,2.0,SOFT,32.0,NaN,Final Stint
2,2018,1,Albert Park Grand Prix Circuit,Lewis Hamilton,Mercedes,58,2,1,21.821,Australian Grand Prix,...,0.017241,44.73482,0.043478,6.754254,HAM,1.0,ULTRASOFT,17.0,19.0,21.821
3,2018,1,Albert Park Grand Prix Circuit,Lewis Hamilton,Mercedes,58,2,1,21.821,Australian Grand Prix,...,0.017241,44.73482,0.043478,6.754254,HAM,2.0,SOFT,39.0,NaN,Final Stint
4,2018,1,Albert Park Grand Prix Circuit,Kimi RÃƒÂ¤ikkÃƒÂ¶nen,Ferrari,58,3,1,21.421,Australian Grand Prix,...,0.017241,45.13482,0.086957,6.818562,RAI,1.0,ULTRASOFT,17.0,18.0,21.421
5,2018,1,Albert Park Grand Prix Circuit,Kimi RÃƒÂ¤ikkÃƒÂ¶nen,Ferrari,58,3,1,21.421,Australian Grand Prix,...,0.017241,45.13482,0.086957,6.818562,RAI,2.0,SOFT,40.0,NaN,Final Stint
6,2018,1,Albert Park Grand Prix Circuit,Daniel Ricciardo,Red Bull,58,4,1,21.440,Australian Grand Prix,...,0.017241,45.11582,0.130435,6.820062,RIC,1.0,SUPERSOFT,25.0,26.0,21.44
7,2018,1,Albert Park Grand Prix Circuit,Daniel Ricciardo,Red Bull,58,4,1,21.440,Australian Grand Prix,...,0.017241,45.11582,0.130435,6.820062,RIC,2.0,SOFT,32.0,NaN,Final Stint
8,2018,1,Albert Park Grand Prix Circuit,Fernando Alonso,McLaren,58,5,1,22.573,Australian Grand Prix,...,0.017241,43.98282,0.173913,6.654571,ALO,1.0,ULTRASOFT,25.0,26.0,22.573
9,2018,1,Albert Park Grand Prix Circuit,Fernando Alonso,McLaren,58,5,1,22.573,Australian Grand Prix,...,0.017241,43.98282,0.173913,6.654571,ALO,2.0,SOFT,32.0,NaN,Final Stint


In [10]:
pg_user = os.getenv("PG_USER")
pg_password = quote_plus(os.getenv("PG_PASSWORD"))  # ← this encodes special chars
pg_host = os.getenv("PG_HOST", "localhost")
pg_port = os.getenv("PG_PORT", "5432")
pg_db = os.getenv("PG_DB1")
csv_db = os.getenv("PG_DB2")
db_path = f"postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{csv_db}"
engine = create_engine(db_path)
# df.to_sql("titanic", engine, index=False)

In [12]:
df.to_sql("f1", engine, index=False)

374

In [11]:
db = SQLDatabase(engine)

In [11]:
print(db.dialect)
print(db.get_usable_table_names())

postgresql
['cancer', 'diabetes', 'f1']


In [12]:
column_names = db.run("SELECT column_name FROM information_schema.columns WHERE table_name = 'f1'")
print(column_names)

[('Season',), ('Round',), ('Humidity_%',), ('Wind_Speed_KMH',), ('Lap Time Variation',), ('Total Pit Stops',), ('Tire Usage Aggression',), ('Fast Lap Attempts',), ('Position Changes',), ('Driver Aggression Score',), ('Stint',), ('Stint Length',), ('Pit_Lap',), ('Laps',), ('Position',), ('TotalPitStops',), ('AvgPitStopTime',), ('Air_Temp_C',), ('Track_Temp_C',), ('Circuit',), ('Driver',), ('Constructor',), ('Country',), ('Tire Compound',), ('Abbreviation',), ('Pit_Time',), ('Race Name',), ('Date',), ('Time_of_race',), ('Location',)]


In [21]:
correct_query = """
SELECT DISTINCT("Driver") 
FROM f1 
WHERE "Position" = 1  -- Changed "POSITION" to "Position"
"""

In [22]:
db.run(correct_query)

"[('Kimi RÃƒÂ¤ikkÃƒÂ¶nen',), ('Oscar Piastri',), ('Valtteri Bottas',), ('Lando Norris',), ('Max Verstappen',), ('Daniel Ricciardo',), ('Pierre Gasly',), ('Esteban Ocon',), ('Lewis Hamilton',), ('Charles Leclerc',), ('Sebastian Vettel',), ('George Russell',), ('Sergio PÃƒÂ©rez',), ('Carlos Sainz',)]"

In [2]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")
print("Environment variables are loaded:", load_dotenv())

Environment variables are loaded: True


In [7]:
# Initialize the LLM - specify your preferred model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # or "gpt-4", "gpt-4-turbo", etc.
    temperature=0.0
)

# Create messages (LangChain message format)
messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="hello")
]

# Get response
response = llm.invoke(messages)
print(response.content)

Hello! How can I assist you today?


In [12]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [13]:
agent_executor.invoke({"input": "Tell me who finished in the top three positions most of the times"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


cancer, diabetes, f1
Invoking: `sql_db_schema` with `{'table_names': 'f1'}`



CREATE TABLE f1 (
	"Season" BIGINT, 
	"Round" BIGINT, 
	"Circuit" TEXT, 
	"Driver" TEXT, 
	"Constructor" TEXT, 
	"Laps" BIGINT, 
	"Position" BIGINT, 
	"TotalPitStops" BIGINT, 
	"AvgPitStopTime" DOUBLE PRECISION, 
	"Race Name" TEXT, 
	"Date" TEXT, 
	"Time_of_race" TEXT, 
	"Location" TEXT, 
	"Country" TEXT, 
	"Air_Temp_C" DOUBLE PRECISION, 
	"Track_Temp_C" DOUBLE PRECISION, 
	"Humidity_%%" DOUBLE PRECISION, 
	"Wind_Speed_KMH" DOUBLE PRECISION, 
	"Lap Time Variation" DOUBLE PRECISION, 
	"Total Pit Stops" DOUBLE PRECISION, 
	"Tire Usage Aggression" DOUBLE PRECISION, 
	"Fast Lap Attempts" DOUBLE PRECISION, 
	"Position Changes" DOUBLE PRECISION, 
	"Driver Aggression Score" DOUBLE PRECISION, 
	"Abbreviation" TEXT, 
	"Stint" DOUBLE PRECISION, 
	"Tire Compound" TEXT, 
	"Stint Length" DOUBLE PRECISION, 
	"Pit_Lap" DOUBLE PRECISION

{'input': 'Tell me who finished in the top three positions most of the times',
 'output': 'The drivers who finished in the top three positions most of the times are:\n\n1. Max Verstappen - 266 times\n2. Lewis Hamilton - 214 times\n3. Valtteri Bottas - 115 times'}